In [ ]:
%load_ext lab_black

In [ ]:
experiment_id = 0
use_gpr = False

In [ ]:
"""Setup the environment."""
from pathlib import Path
import os
import sys

import arviz as az
import bambi as bmb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

# Add utility package to path
sys.path.append(os.environ.get("BRAINWEB_TDCS_CODE_DIR", "../code"))
from brainweb_tdcs import EXPERIMENTS
from brainweb_tdcs.plot import (
    display_side_by_side,
    plot_subject,
    plot_posterior,
)

# Set path data directory
if "BRAINWEB_TDCS_DATA_DIR" not in os.environ:
    os.environ["BRAINWEB_TDCS_DATA_DIR"] = str((Path.cwd() / "../data").resolve())

# Set the random seed
RANDOM_SEED = 1234

In [ ]:
"""Set some constants."""
CAPTIONS = {
    "e": "Mean absolute magnitude of the\nelectric field",
    "e_r": "Mean absolute magnitude of the\nnormal component of electric field",
}
LABELS = {
    "e": "$| \mathbf{e} |$ (mVm$^{-1}$)",
    "e_r": "$| \mathbf{e}_r |$ (mVm$^{-1}$)",
}
FUNC_DICT = {
    "mean": np.mean,
    "std": np.std,
    "2.5%": lambda x: np.percentile(x, 2.5),
    "97.5%": lambda x: np.percentile(x, 97.5),
}

In [ ]:
"""Select the experiment."""
experiment = EXPERIMENTS[experiment_id]
display(
    Markdown(
        "# Evaluate the effect of the head geometry "
        f"({experiment.roi}, {experiment.montage} electrodes montage)"
    )
)

In [ ]:
display(
    HTML(
        f"""
<div style="background: #efefef; color: #5f5f5f; padding: 15pt;">
    <b style="display: inline-block; width: 120pt;">Region of interest</b> {experiment.roi.long_name} ({experiment.roi})</br>
    <b style="display: inline-block; width: 120pt;">Electrodes montage</b> {experiment.montage} ({"Bipolar" if experiment.is_bipolar else "Unipolar"})</br>
    </br>
    In this notebook, we evaluate the effect of the head geometry on the electric field computed in the region of interest resulting from the stimulation using the {'realistic distribution' if use_gpr else 'uniform distribution'}.
</div>
"""
    )
)

## Data

First, we load the data corresponding to this experiment and display the measurements of the average absolute mangitude of the electric field $\bar{| \pmb{e} |}$ and of its component normal to the cortical surface $\bar{| \pmb{e}_r |}$.

In [ ]:
"""Load the experiment results."""
if use_gpr:
    data = experiment.get_gpr_data()[["sub", "e", "e_r"]]
else:
    data = experiment.get_data()[["sub", "e", "e_r"]]

In [ ]:
"""Plot mean absolute magnitude of the electric field and radial component."""
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
for i, voi in enumerate(("e", "e_r")):
    plot_subject(axs[i], voi, experiment, data, LABELS[voi], CAPTIONS[voi])
plt.show()
columns = ["mean", "std", "min", "25%", "75%", "max"]
display_side_by_side(
    [
        data.groupby("sub").describe()["e"][columns],
        data.groupby("sub").describe()["e_r"][columns],
    ],
    ["", ""],
)

## Models

To compare the measurements acquired with the different subject head geometries, we use multiple Bayesian linear models:
$$
Y_i \sim \mathcal{N}(\mu_i, \sigma_i^2),
$$
where $Y_i$ is the dependent variable we focus on and
$$
\mu_i = \alpha_i + \sum_{s \neq i}^{20} \beta_s \cdot X_s + \varepsilon
$$
with $\beta_s$ the slopes, $X_s$ the independent variables and $\varepsilon$ the error term. We build these models for each subject so that we obtain the pairwise comparison of all the subjects.

For all these models, weakly informative priors are set (Westfall, 2017). They are then fitted using the NUTS sampler (Hoffman *et al.*, 2011).

In [ ]:
"""Define model storages."""
models = dict(e=None, e_r=None)
fits = dict(e=None, e_r=None)
summaries = dict(e=None, e_r=None)
rope_vs_hdi = dict(e=dict(), e_r=dict())

In [ ]:
"""Define indices of the subject to set the references of the models."""
subs = data["sub"].values
sub_ids = data["sub"].unique()
for i, sub_id in enumerate(sub_ids):
    ids = np.zeros(data.shape[0])
    for j in range(sub_ids.size):
        ids[subs == sub_ids[(i + j) % sub_ids.size]] = (
            j if (i + j) % sub_ids.size >= i else -1
        )
    data[f"sub{sub_id}"] = ids

In [ ]:
"""Define models."""
for voi in ("e", "e_r"):
    models[voi] = [
        bmb.Model(f"{voi} ~ C(sub{i})", data[data[f"sub{i}"] >= 0])
        for i in sub_ids[:-1]
    ]

In [ ]:
"""Fit models."""
for voi in ("e", "e_r"):
    fits[voi] = [
        model.fit(draws=1000, chains=4, random_seed=RANDOM_SEED)
        for model in models[voi]
    ]

In [ ]:
"""Summarize results."""
for voi in ["e", "e_r"]:
    summaries[voi] = []
    for i, sub_id in enumerate(sub_ids[:-1]):
        indices = [
            "α",
            *[
                f"β_sub-{sub_ids[(i + j + 1) % sub_ids[i:].size]:02d}"
                for j in range(sub_ids[i:].size - 1)
            ],
            "σ",
        ]
        summary = az.summary(fits[voi][i], stat_funcs=FUNC_DICT, extend=False)
        summary.index = indices
        summaries[voi].append(summary)

In [ ]:
"""Plot betas for the models."""
n_subs = sub_ids.size
for voi in ("e", "e_r"):
    fig, axs = plt.subplots(n_subs, n_subs, figsize=(40, 40), sharex="all")
    rope_vs_hdi[voi] = []
    for i, a_id in enumerate(sub_ids[:-1]):
        betas = summaries[voi][i].index[1:-1]
        rvhs = []
        for j, beta in enumerate(betas):
            rvh = plot_posterior(
                axs[i][(i + j + 1) % n_subs],
                experiment,
                fits[voi][i],
                summaries[voi][i],
                f"C(sub{a_id})",
                "",
                "",
                j,
                summary_param=beta,
                show_zero=True,
                beta_suffix=beta.split("_")[1],
                rope_width=0.1 * data[voi].std(),
            )
            rvhs.append(rvh)
        rope_vs_hdi[voi].append(rvhs)
    fig.tight_layout()
    plt.show()

In [ ]:
"""Get matrices."""


def display_df(df, title):
    display(
        HTML(
            df.style.set_table_attributes("style='display: inline'")
            .set_caption(title)
            .format(precision=2)
            ._repr_html_()
        )
    )


metrics = dict(e=dict(), e_r=dict())
for voi in ["e", "e_r"]:
    display(Markdown(rf"## $| {voi} |$"))
    for metric in ["mean", "std", "2.5%", "97.5%"]:
        mat = np.zeros((n_subs, n_subs))
        for i, a_id in enumerate(sub_ids[:-1]):
            mat[i, i + 1 :] = summaries[voi][i][metric][1:-1]
        metrics[voi][metric] = pd.DataFrame(mat, index=sub_ids, columns=sub_ids)
        display_df(metrics[voi][metric], metric.capitalize())
    for j, metric in enumerate(["HDI-ROPE", "ROPE-HDI"]):
        mat = np.zeros((n_subs, n_subs))
        for i, a_id in enumerate(sub_ids[:-1]):
            mat[i, i + 1 :] = [v[j] for v in rope_vs_hdi[voi][i]]
        metrics[voi][metric] = pd.DataFrame(mat, index=sub_ids, columns=sub_ids)
        display_df(metrics[voi][metric], metric.capitalize())

## References

- Hoffman *et al.* (2011) - https://arxiv.org/abs/1111.4246
- Westfall (2017) - https://arxiv.org/abs/1702.01201